In [1]:
# imports
import pandas as pd
from functions import *
import requests
from dotenv import dotenv_values
import time

In [2]:
# read in the data
songs = pd.read_csv('data/d_t100_5c_songs.csv', header=0, index_col=False)

In [3]:
# show the last rows
songs.tail(3)

,title,artist,track_id
13142,No Hay Nadie Más,Sebastian Yatra,0U8WdltGpu3OC0ma9yLmjX
13143,Higher We Go - Intro,Migos,43dOwOq0mB4i6j4SpcSkqC
13144,Too Much Jewelry,Migos,4loAa1lBkE83qJXU1ddmCV


In [4]:
# create a list with the track id of each song
songs_list = songs['track_id'].tolist()

In [5]:
# show the first 5
songs_list[0:5]

['5knuzwU65gJK7IF5yJsuaW',
 '0JiVRyTJcJnmlwCZ854K4p',
 '2nYpbaLykC2cjU8Yt5KF1v',
 '5aAx2yezTd8zXrkmtKl66Z',
 '7BKLCZ1jbUBVqRi2FVlTVw']

In [6]:
# show the last 5
songs_list[13140:13145]

['4jbmgIyjGoXjY01XxatOx6',
 '7ybzAh1zEWoVn880qZ15XL',
 '0U8WdltGpu3OC0ma9yLmjX',
 '43dOwOq0mB4i6j4SpcSkqC',
 '4loAa1lBkE83qJXU1ddmCV']

In [7]:
# get authorization token valid for one hour
client_id = dotenv_values('.env')['spotify_id']
client_secret = dotenv_values('.env')['spotify_secret']

data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}
response = requests.post('https://accounts.spotify.com/api/token', data=data)
print(response.json())

{'access_token': 'BQC90m0P-B2wv_9dVT5ltQbq3JmLkPb56RIPTXjY5s9y8ygTp86x-rVL40WHjDbNgmhAZ0rvT-0RuljrrLrkaH7oZ4uX3rkQKk6eAJFlu7M684xUqEA', 'token_type': 'Bearer', 'expires_in': 3600}


In [8]:
# run a test request for one song
token = 'Bearer' + ' ' + 'BQC90m0P-B2wv_9dVT5ltQbq3JmLkPb56RIPTXjY5s9y8ygTp86x-rVL40WHjDbNgmhAZ0rvT-0RuljrrLrkaH7oZ4uX3rkQKk6eAJFlu7M684xUqEA'
headers = {
    'Authorization': token,
}
final_url = 'https://api.spotify.com/v1/tracks?ids=' + '2nYpbaLykC2cjU8Yt5KF1v'
response = requests.get(final_url, headers=headers)
print(response.json())

{'tracks': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7qXzy6c5RWT0XlVQcOBIDG'}, 'href': 'https://api.spotify.com/v1/artists/7qXzy6c5RWT0XlVQcOBIDG', 'id': '7qXzy6c5RWT0XlVQcOBIDG', 'name': 'Mark Forster', 'type': 'artist', 'uri': 'spotify:artist:7qXzy6c5RWT0XlVQcOBIDG'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/3PYi5cqKxVEgc1HTMvzStf'}, 'href': 'https://api.spotify.com/v1/albums/3PYi5cqKxVEgc1HTMvzStf', 'id': '3PYi5cqKxVEgc1HTMvzStf', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27329cd01a46be6d3b100e22739', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e0229cd01a46be6d3b100e22739', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d0000485129cd01a46be6d3b100e22739', 'width': 64}], 'name': 'TAPE', 'release_date': '2016-06-03', 'release_date_precision': 'day', 'total_tracks': 14, 'type': 'album', 'uri

In [9]:
# run reuqest for 1000 songs
url = 'https://api.spotify.com/v1/tracks?ids='
updated_songs = pd.DataFrame([])
steps = songs_list[3001:4001]
for i in steps:
    url_f = url + i
    r = requests.get(url_f, headers=headers)
    df_temp = r.json()
    df_temp_2 = pd.json_normalize(df_temp, 
                                        sep="_", 
                                        record_path="tracks", 
                                        meta=['album', ['release_date', 'popularity']], 
                                        errors='ignore')
    updated_songs = pd.concat([updated_songs, df_temp_2], ignore_index=True)
    time.sleep(2)

In [10]:
# count the returned 
updated_songs.id.count()

1000

In [11]:
# export the data
save(updated_songs)

/Users/inthesea/Downloads/df.csv
